# Guild AI Interactive Python Quick Start

This Notebook mirrors the steps in [Guild AI Quick Start](https://guild.ai/docs/start/).

Guild AI is a tool for running, tracking, and comparing machine learning experiments. For more on Guild AI, visit https://guild.ai.

In this example, we show how Guild is used to run an unmodified Python function as a training operation. It uses a Pythonic interface defined in the module `guild.ipy`.

This notebook covers the following topics:

- Installation
- Experiment basics
    - Create a mock training function
    - Generate a run with Guild
    - Examine the run
    - Generate a second run
    - Compare runs
- Hyperparameter search and optimization
    - Grid search
    - Random search
    - Bayesian optimization

## Installation

Before continuing, install Guild AI:

    $ pip install guildai
    
Install either `tensorflow` or `tensorflow-gpu` depending on your system capability:

    $ pip install tensorflow # (or tensorflow-gpu if your system has a GPU)
    
TensorFlow is required for TensorBoard support. Your training code does not need to use TensorFlow. This requirement will be removed in versions of Guild starting with `0.7`.
    
<span style="color:#600;font-weight:600">IMPORTANT</span> If you are installing Guild AI 0.6.3, you must manually install a missing requirement (this bug is fixed in 0.6.4, which will be released in late May):

    $ pip install click
    
## Contents
    
This Notebook has two parts.

It features the module `guild.ipy`, which is a Python interface that can be used interactively in Notebooks or by your own Python code. 

## Notebooks config

Modify the variables below to change Notebook configuration.

`GULID_HOME` is the location of generated runs. By default it is the location subdirectory "guild-env". Note that we initialize this directory below by permanently deleting any runs it contains before proceeding. If you don't want to delete runs in `GUILD_HOME`, set `DELETE_RUNS_ON_INIT` to `False` below.

In [ ]:
GUILD_HOME = "guild-env"

`DELETE_RUNS_ON_INIT` determines whether or not runs are initially deleted from `GUILD_HOME` below. As this Notebook is for demonstration purposes, it's usually a good idea to delete any existing runs before proceeding. To prevent any runs from being deleted, set `DELETE_RUNS_ON_INIT` to `False`.

In [ ]:
DELETE_RUNS_ON_INIT = True

## Guild interactive Python API

Guild AI functionality is available through a Notebook compatible interface defined in `guild.ipy`.

Import this module as `guild`:

In [ ]:
import guild.ipy as guild

<span style="color:#600;font-weight:600">IMPORTANT</span> If you get an error **No module named 'click'** install `click` manually by running `pip install click` in your Notebook environment.

## Initialize Guild Home

Guild home is a directory containing the runs that Guild generates. This Notebook uses the directory defined by `GUILD_HOME`. Ensure that the directory exists and use `set_guild_home()` to set the value. 

In [ ]:
import os

if not os.path.exists(GUILD_HOME):
    os.mkdir("guild-env")
    
guild.set_guild_home("guild-env")

Clear the director of any runs.

In [ ]:
if DELETE_RUNS_ON_INIT:
    deleted = guild.runs().delete(permanent=True)
    print("Deleted %i run(s)" % len(deleted))

## Mock training function

Create a mock training script. This function doesn’t actually train anything, but simulates the training process of accepting hyperparameters as inputs and generating a loss.

In [ ]:
import numpy as np

def train(x=0.1, noise=0.1):
    loss = (np.sin(5 * x) * (1 - np.tanh(x ** 2)) + np.random.randn() * noise)
    print("loss: %f" % loss)

[Function credit: *skopt API documentation*](https://scikit-optimize.github.io/)

Run the mock training function a couple times to see how it works.

**NOTE** The function uses a random component to simulate training "noise". This causes the results to differ across runs.

In [ ]:
print("Trial 1:")
train(x=-2.0)

In [ ]:
print("Trial 2:")
train(x=0.0)

In [ ]:
print("Trial 3:")
train(x=2.0)

## Generate a run

Run the mock train function using Guild:

In [ ]:
run, return_val = guild.run(train)

The `run()` function returns a tuple of the Guild run and the return value of the function.

In this case, our mock training function doesn't return a value.

In [ ]:
print(return_val)

The `run` variable is a Python object that represents the run.

## Examine the run

We can work with the run object directly to get information about it.

Each run has a uniqiue ID:

In [ ]:
run.id

Each run is associated with a unique directory, which includes the ID. The run directory contains metadata and output associated with the run.

In [ ]:
run.dir

Our mock training script doesn't generate any files, so the run directory is empty, with the exception of a `.guild` subdirectory.

In [ ]:
os.listdir(run.dir)

The `.guild` subdirectory contains run output and metadata. Outputs include the output printed to the script during the operation (`output`) and any scalars logged (`events.*`). Guild logs scalar output using TensorFlow event files, which can be read using TensorBoard.

In [ ]:
os.listdir(os.path.join(run.dir, ".guild"))

Run metadata includes *attributes*. You can list the attribute names using `attr_names()`:

In [ ]:
run.attr_names()

Read an attribute value using `get()`. For example, to read the *flags* attribute, use:

In [ ]:
run.get("flags")

Flags are values provided to the function. In this case, both values are defined as function keyword default values. Later you run `train` using explicit flag values.

Each run has a *status*, which indicates if the run is still running and whether or not it completed successfully or terminated with an error.

In [ ]:
run.status

## List runs

List runs using `guild.runs()`:

In [ ]:
runs = guild.runs()
runs

Each time you run an operation, the run appears in the data frame generated by `runs()`.

## Print run info

From `runs`, we can print information for the latest run using `runs.info()`.

In [ ]:
runs.info()

This information reflects the information you saw in the prior section.

## Run scalars

Run results are recorded as *scalars*. You can list scalars in two ways.

First, you can specify the `scalars` flag to `info()`:

In [ ]:
runs.info(scalars=True)

Second, you can get a data frame containing scalars using the `scalars()` method.

In [ ]:
scalars = runs.scalars()
scalars

<span style="color:#600;font-weight:600">IMPORTANT</span> If this listing is empty, ensure that you have TensorFlow installed (see above for instructions).

Guild stores aggregates of each tag, including *first*, *last*, *max*, *min*, and *average*.

You can use the various facilities in data frame to get scalar values.

In [ ]:
scalars.query("(run == '%s') and (tag == 'loss')" % run.id)["last_val"]

## Generate a second run

Run train again with different flags along with a run *label*:

In [ ]:
_ = guild.run(train, x=0.2, _label="run 2")

List runs:

In [ ]:
guild.runs()

Note that the latest run contains the label "run 2" as specified in the `run()` call.

## Compare runs

Use `compare()` to generate a data frame containing both flags and scalars.

In [ ]:
guild.runs().compare()

## Grid search

Grid search &mdash; also referred to as a parameter sweep &mdash; is a form of hyperparameter tuning that uses exhaustive search over a manually defined set of hyperparameter values.

To perform a grid search in Guild, provide a list of values to use for any given flag. If you specify lists for multiple flags, Guild runs trials for each possible flag value combination.

Run `train()` using a range of values:

In [ ]:
_ = guild.run(train, x=[-0.5,-0.4,-0.3,-0.2,-0.1])

This command generates five trials, one for each specified value of `x`.

Use `runs().compare()[:5]` to compare the last five results:

In [ ]:
guild.runs().compare()[:5]

View the top-three results by loss:

In [ ]:
guild.runs().compare().sort_values(by="loss")[:3]

Based on our mock training function, the "best" result (i.e. the run with the lowest *loss*) should be the run where `x` is close to `-0.3`. Because there's a random component (i.e. the `noise` parameter) your results may show best results with different values for `x`.

Below is an image that plots *loss* for values of *x*, showing the lowest loss where x is approximately `-0.3`.

<img src="bayesian-optimization.png" style="margin-left:0">

[Image credit: *Bayesian optimization with skopt*](https://scikit-optimize.github.io/notebooks/bayesian-optimization.html)

## Random search

Random search is a method used in machine learning to explore hyperparameter spaces at random.

To run a series of runs using random values over a specified range, use the `slice` function to specify a range for a flag. To specify the number of trials, specify `_max_trials`. The default number of trials is `20`.

Run train five times with random values of `x` over the range `-2.0` to `2.0`.

In [ ]:
_ = guild.run(train, x=slice(-2.0,2.0), _max_trials=5)

Compare the results again.

In [ ]:
guild.runs().compare()

List the three runs with the lowest loss:

In [ ]:
guild.runs().compare().sort_values(by="loss")[:3]

## Bayesian optimization

Bayesian optimizers use light weight models as surrogates for the target model — surrogates that can be evaluated quickly to recommend likely optimal hyperparameter values — and update those models using results from real trials.

Run `train` with Guild’s built-in Bayesian optimizer, which uses Gaussian processes. As with the earlier random search, we use Python's slice function to specify the range for `x` over which to search.

**NOTE:** The argument `bayesian` in the command is an alias for `gp`, which is a Bayesian optimizer that uses Gaussian processes. Guild supports three Bayesian optimizers: gp, forest and gbrt.

In [ ]:
_ = guild.run(train, x=slice(-2.0,2.0), _optimizer="bayesian", _max_trials=10)

View the top 10 results:

In [ ]:
guild.runs().compare().sort_values(by="loss")[:10]

## Next steps

This Notebook covers basic functionality provided by the `guild.ipy` module. The module lets you run functions and capture results as unique runs. You can view and compare run results using various module functions.

Generated runs can be further managed using the Guild command line interface. For more information, refer to [Guild AI Quick Start](https://guild.ai/docs/start/).